In [145]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics


import pandas as pd
import numpy as np
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.utils.data as data
from PIL import Image
import torch.nn.functional as F

In [4]:
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"GPU is available with {num_gpus} device(s).")
    
    # Get the name of the current GPU
    current_gpu = torch.cuda.get_device_name(0)  # Assuming the first GPU is used
    print(f"Current GPU: {current_gpu}")
else:
    print("GPU is not available. Using CPU.")

GPU is not available. Using CPU.


In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the first available GPU
else:
    device = torch.device("cpu")  # If no GPU is available, use CPU

In [72]:
current_directory = os.getcwd()
models_dir = current_directory + '/models'
os.makedirs(models_dir, exist_ok=True)
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [123]:
class SolarPanelDataset(data.Dataset):

    #data_dir: The directory containing the image files
    #image_shape: The shape of the images (height, width, channels)
    #augmentation_copies: The number of copies of each image to create for data augmentation

    def __init__(self, df, data_dir, image_shape=(3, 256, 256), augmentation_copies=3):

        self.copies = 1 + augmentation_copies
        self.image_shape = image_shape
        self.df = df #df: A pandas DataFrame containing the image filenames and labels
        self.labels = []

        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
        ])

        self.augmentation_transform = self.make_augmentation_transform()
        self.dataset_size = len(df)
        self.images = torch.zeros((self.dataset_size, *self.image_shape), dtype=torch.float32)

        for i, img_name in enumerate(self.df['original_title']):
            img_path = os.path.join(data_dir, img_name)
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            self.images[i] = image
            label = df.iloc[i]['loss_percentage']
            self.labels.append(label)  # Add corresponding label here

    def __len__(self):
        return len(self.images)*self.copies

    def make_augmentation_transform(self):
        return transforms.Compose([
            transforms.RandomResizedCrop((self.image_shape[1], self.image_shape[2])),
            transforms.RandomHorizontalFlip(p=0.2),
            transforms.RandomVerticalFlip(p=0.2),
            transforms.RandomRotation(45),
            transforms.ColorJitter(brightness=(0.3, 2), contrast=0, saturation=0, hue=0),
            #transforms.RandomGrayscale(p=0.1)
        ])
        
    def __getitem__(self, idx):
        if (idx >= len(self.images) ):
            image = self.images[idx % len(self.images)]
            label = self.labels[idx % len(self.labels)]
            return self.augmentation_transform(image), torch.tensor(label, dtype=torch.float32)
            
            
        image = self.images[idx]
        label = self.labels[idx]
        return image, torch.tensor(label, dtype=torch.float32)

In [124]:
class ProfessorsCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(59536, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [201]:

# Function to save the model
def saveModel(model:nn.Module, path):
    torch.save(model.state_dict(), path)

# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(model: torch.nn.Module,
          dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          metric: torchmetrics.Metric,
          device: torch.device,
          num_epochs,
          path_model,
          report: pd.DataFrame,
          verbatim):
    model.train()
    model.to(device)
    metric.to(device)
    best_accuracy = 0.0
    best_loss = 0.0
    best_epoch = 0
    metric_algorithm = metric.__class__.__name__
    loss_algorithm = loss_fn.__class__.__name__
    best_metric = 100
    for epoch in range(num_epochs):  # loop over the dataset multiple times

        train_acc = 0.0
        train_loss = 0.0
        for ibatch, (images, labels) in enumerate(dataloader, 0):
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            y_pred = model(images)
            y_pred = y_pred.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]
            loss = loss_fn(y_pred, labels)
            train_loss += loss.item()

            metric(y_pred, labels)
            loss.backward()
            optimizer.step()

            print(loss)
            # Calculate and accumulate accuracy metric across all batches
            #y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            #train_acc += (y_pred_class == labels).sum().item()/len(y_pred)

        # Adjust metrics to get average loss and accuracy per batch
        train_loss = train_loss / len(dataloader)

        # we want to save the model if the accuracy is the best

        path = "./myModel_" +str(epoch)+ ".pth"
        #saveModel(model, path = path)

        batch_metric = metric.compute()
        report_line = {'epoch': epoch,
                       'batch_loss': train_loss,
                       'metric_algorithm': metric_algorithm,
                       'batch_metric': batch_metric.item(),
                       'loss_algorithm': loss_algorithm
                      }
        report = pd.concat([report, pd.DataFrame([report_line])], ignore_index=True)

        if batch_metric < best_metric:
            path = str(path_model + '/bestModel.pth')
            saveModel(model, path = path)
            best_loss = train_loss
            best_metric = batch_metric
            best_epoch = epoch
            if verbatim:
              print('Best Epoch #', epoch,' Loss=', best_loss, " Accu=", best_accuracy )

    return best_loss, best_accuracy, best_epoch, report

In [202]:
model = ProfessorsCNN()
metric = torchmetrics.MeanAbsolutePercentageError()
dataset = SolarPanelDataset(train_df.sample(200), '/home/victor/mestrado/cv/project/CV-Project/SolarPanelSoilingImageDataset/Solar_Panel_Soiling_Image_dataset/PanelImages/')
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
loss_fn = torch.nn.MSELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
num_epochs = 2
path_model = models_dir + f'/{model.__class__.__name__}_{loss_fn.__class__.__name__}'
os.makedirs(path_model, exist_ok=True)
report = pd.DataFrame()
_, _, _, report = train(model, train_dataloader, loss_fn, optimizer, metric, device, num_epochs, path_model, report, verbatim=True)

/home/victor/mestrado/cv/virtualenv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor(0.2195, grad_fn=<MseLossBackward0>)
tensor(0.2211, grad_fn=<MseLossBackward0>)
tensor(0.1480, grad_fn=<MseLossBackward0>)
tensor(0.1420, grad_fn=<MseLossBackward0>)
tensor(0.1559, grad_fn=<MseLossBackward0>)
tensor(0.1120, grad_fn=<MseLossBackward0>)
tensor(0.0523, grad_fn=<MseLossBackward0>)
tensor(0.0735, grad_fn=<MseLossBackward0>)
tensor(0.0942, grad_fn=<MseLossBackward0>)
tensor(0.0809, grad_fn=<MseLossBackward0>)
tensor(0.0910, grad_fn=<MseLossBackward0>)
tensor(0.0952, grad_fn=<MseLossBackward0>)
tensor(0.0922, grad_fn=<MseLossBackward0>)
Best Epoch # 0  Loss= 0.12136906385421753  Accu= 0.0
tensor(0.1021, grad_fn=<MseLossBackward0>)
tensor(0.1050, grad_fn=<MseLossBackward0>)
tensor(0.1021, grad_fn=<MseLossBackward0>)
tensor(0.0931, grad_fn=<MseLossBackward0>)
tensor(0.0796, grad_fn=<MseLossBackward0>)
tensor(0.0861, grad_fn=<MseLossBackward0>)
tensor(0.0743, grad_fn=<MseLossBackward0>)
tensor(0.0858, grad_fn=<MseLossBackward0>)
tensor(0.1127, grad_fn=<MseLossBackward0>)
t

In [210]:
# Function to test the model with the test dataset and print the accuracy for the test images
def test(model: torch.nn.Module,
         dataloader: torch.utils.data.DataLoader,
         loss_fn: torch.nn.Module,
         metric: torchmetrics.Metric,
         device: torch.device,
         verbatim = True):

    model.eval()
    metric = metric.to(device)

    # Setup test loss and test accuracy values
    test_loss = 0
    test_metric = 100
    pred_labels = []
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            y_pred = model(images)
            y_pred = y_pred.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]
            loss = loss_fn(y_pred, labels)
            test_loss += loss.item()
            metric.update(y_pred, labels)

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_metric = metric.compute()

    if verbatim:
      print("Loss =", test_loss, f'  Metric ({metric.__class__.__name__})=', test_metric.item())
    return pred_labels, test_loss, test_metric

In [211]:
metric = torchmetrics.MeanAbsolutePercentageError()
test_dataset = SolarPanelDataset(test_df.sample(200), '/home/victor/mestrado/cv/project/CV-Project/SolarPanelSoilingImageDataset/Solar_Panel_Soiling_Image_dataset/PanelImages/')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)
test(model, test_dataloader, loss_fn, metric, device, verbatim=True)

Loss = 0.08562727957963943   Metric (MeanAbsolutePercentageError)= 18.312280654907227


([], 0.08562727957963943, tensor(18.3123))

In [33]:

def tensor_to_image(tensor):

    """
    Converts a tensor to an image.

    Args:
        tensor: A tensor of shape (C, H, W).

    Returns:
        An image.
    """

    image = tensor.cpu().detach().numpy()
    image = image.transpose(1, 2, 0)  # Convert from (C, H, W) to (H, W, C)
    image = (image * 255).astype('uint8')  # Convert pixel values from [0, 1] to [0, 255]

    return image
